# CO460 - Deep Learning - Lab exercise 5

## Introduction

In this exercise, you will experiment with a GAN and a Conditional GAN architecture.
You are asked to:

1.  Experiment with the architectures
2.  Define the training strategy
3.  Investigate and implement sampling and interpolation in the latent space.

In [0]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image, make_grid
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

# utils
def interpolate(z1, z2, num=11):
    Z = np.zeros((z1.shape[0], num))
    for i in range(z1.shape[0]):
        Z[i, :] = np.linspace(z1[i], z2[i], num)
    return Z

def denorm_for_tanh(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

def denorm_for_sigmoid(x):
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

def denorm_for_binary(x):
    x = x.clamp(0, 1)
    x = x>0.5
    x = x.view(x.size(0), 1, 28, 28)
    return x

def show(img):
    if torch.cuda.is_available():
        img = img.cpu()
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))

### Device selection

In [0]:
GPU = True
device_idx = 0
if GPU:
    device = torch.device("cuda:" + str(device_idx) if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
print(device)

cuda:0


### Reproducibility

In [0]:
# We set a random seed to ensure that your results are reproducible.
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
torch.manual_seed(0)

In [0]:
transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize(mean=(0.5,), std=(0.5,))
])

train_dat = datasets.MNIST(
    "data/", train=True, download=True, transform=transform
)
test_dat = datasets.MNIST("./data/", train=False, transform=transform)

denorm = denorm_for_tanh

if not os.path.exists('./cGAN'):
    os.mkdir('./cGAN')
    
if not os.path.exists('./GAN'):
    os.mkdir('./GAN')

### Hyper-parameter selection

In [0]:
"""
TODO: Define here your hyperparameters
"""

num_epochs = None
batch_size = None
learning_rate = None
noise_dim = None


in_dim = np.prod(train_dat[0][0].shape)
out_shape = train_dat[0][0].shape
sample_interval = 5

In [0]:
train_loader = DataLoader(train_dat, batch_size, shuffle=True, num_workers=16)
test_loader = DataLoader(test_dat, batch_size, shuffle=False, num_workers=16)

### Define the model

In [0]:
class Generator(torch.nn.Module):

    def __init__(self):
        super(Generator, self).__init__()
        """
        TODO: Layer definitions for the Generator
        """
        
    def forward(self, x):
    
        """
        TODO: Generator pipeline. Your output should have the same dimensions as the real images
        """
            
        return x
    

class Discriminator(torch.nn.Module):

    def __init__(self):
        super(Discriminator, self).__init__()
        """
        TODO: Layer definitions for the Discriminator
        """

    def forward(self, x):
        """
        TODO: Discriminator pipeline. Your output should have only one dimension.
        """
            
        return x

In [0]:
generator = Generator()
discriminator = Discriminator()

### Define Loss function

In [0]:
criterion = nn.BCELoss(reduction='mean')
def loss_function(out, label):
    loss = criterion(out, label)
    return loss

### Initialize Model and print number of parameters

In [0]:
generator = generator.to(device)
discriminator = discriminator.to(device)

params = sum(p.numel() for p in generator.parameters() if p.requires_grad)
print("Total number of generator parameters is: {}".format(params))  # what would the number actually be
print(generator)

params = sum(p.numel() for p in discriminator.parameters() if p.requires_grad)
print("Total number of discriminator parameters is: {}".format(params))  # what would the number actually be
print(discriminator)

### Choose and initialize optimizer

In [0]:
optimizer_G = torch.optim.Adam(generator.parameters(), lr=learning_rate)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=learning_rate)

### Pick a noise distribution

In [0]:
def generate_noise(batch_size, noise_dim):
    """
    TODO: Define here your noise distribution (probably gaussian or uniform)
    """
    return noise                        

 ### Train

In [0]:
g_losses = []
d_losses = []
generator.train()
discriminator.train()
num_epochs = 200 
fixed_noise = generate_noise(batch_size, noise_dim)

for epoch in range(num_epochs):
    g_loss_epoch = 0
    d_loss_epoch = 0 
    for batch_idx, data in enumerate(train_loader):
        
        """
        TODO: Define here your training strategy
        """
        d_loss_epoch += d_loss.item()
        g_loss_epoch += g_loss.item()


    print('epoch [{}/{}], generator loss:{:.4f}'
          .format(epoch + 1, num_epochs, g_loss_epoch / len(train_loader)))
    g_losses.append(g_loss_epoch/ len(train_loader))
    print('epoch [{}/{}], discriminator loss:{:.4f}'
          .format(epoch + 1, num_epochs, d_loss_epoch / len(train_loader)))
    d_losses.append(d_loss_epoch/ len(train_loader))
    if epoch % sample_interval == 0:
        save_image(denorm(generator(fixed_noise.to(device))).cpu().float(), './GAN/samples_epoch_{}.png'.format(epoch),nrow = 8)
    torch.save(generator.state_dict(), './GAN/generator.pth')
    torch.save(discriminator.state_dict(), './GAN/discriminator.pth')
    
np.save('./GAN/generator_losses.npy', np.array(g_losses))
np.save('./GAN/discriminator_losses.npy', np.array(d_losses))

### Loss curves

In [0]:
import matplotlib.pyplot as plt
generator_losses = np.load('./GAN/generator_losses.npy')
plt.plot(list(range(0,generator_losses.shape[0])), generator_losses)
plt.title('Generator Loss')
plt.show()

import matplotlib.pyplot as plt
discriminator_losses = np.load('./GAN/discriminator_losses.npy')
plt.plot(list(range(0,discriminator_losses.shape[0])), discriminator_losses)
plt.title('Discriminator Loss')
plt.show()

### Sampling

In [0]:
generator.load_state_dict(torch.load("./GAN/generator.pth"))

"""
TODO: Sample from the noise distribution
"""

"""
TODO: Do a linear interpolation in the latent space between two noise vectors 
      and generate all the intermediate samples
"""


## Conditional GAN

The concept of conditional GAN: 

* $\mathbf{x}$ refers to a train datum
* $\mathbf{y}$ refers to the corresponding label
* $\mathbf{z}$ refers to random noise vector
<img src="./imgs/cGAN.png" width="500" />

### Hyper-parameter selection

In [0]:
"""
TODO: Define here your hyperparameters
"""

num_epochs = None
batch_size = None
learning_rate = None

num_workers = 16
num_classes = 10
sample_interval = 5

### Define the dataloaders

In [0]:
train_loader = DataLoader(train_dat, batch_size, shuffle=True, num_workers=num_workers)
test_loader = DataLoader(test_dat, batch_size, shuffle=False, num_workers=num_workers)
total_step = len(train_loader)

it = iter(test_loader)
sample_inputs, _ = next(it)

in_dim = sample_inputs.shape[-1] * sample_inputs.shape[-2]

### Define the Generator and the Discriminator

In [0]:
class Generator(torch.nn.Module):

    def __init__(self):
        super(Generator, self).__init__()
        """
        TODO: Layer definitions for the Generator
        """
        
    def forward(self, x):
    
        """
        TODO: Generator pipeline. Your output should have the same dimensions as the real images
        """
            
        return x
    
  
class Discriminator(torch.nn.Module):

    def __init__(self):
        super(Discriminator, self).__init__()
        """
        TODO: Layer definitions for the Discriminator
        """

    def forward(self, x):
        """
        TODO: Discriminator pipeline. Your output should have only one dimension.
        """
            
        return x

In [0]:
generator = Generator(latent_dim, in_dim)
discriminator = Discriminator(in_dim)

### Define loss function

In [0]:
criterion = nn.BCELoss(reduction='mean')
def loss_function(out, label):
    loss = criterion(out, label)
    return loss

### Initialize Model and print number of parameters for both G and D

In [0]:
generator = generator.to(device)
discriminator = discriminator.to(device)
g_params = sum(p.numel() for p in generator.parameters() if p.requires_grad)
d_params = sum(p.numel() for p in discriminator.parameters() if p.requires_grad)
print("The number of parameters for G is: {}".format(g_params))
print("The number of parameters for D is: {}".format(d_params))
print("The total number of parameters is: {}".format(g_params + d_params))

### Choose and initialize optimizer

In [0]:
g_optimizer = torch.optim.Adam(generator.parameters(), lr=learning_rate)
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=learning_rate)

### Train

In [0]:
generator.train()
discriminator.train()
g_losses = []
d_losses = []

fixed_noise = torch.rand(num_classes, latent_dim).to(device)
fixed_labels = np.arange(num_classes)
fixed_labels = (torch.from_numpy(fixed_labels)).type(torch.LongTensor)
fixed_labels = fixed_labels.to(device)
fixed_labels_one_hot = torch.zeros(num_classes, num_classes).to(device)
fixed_labels_one_hot.scatter_(1, fixed_labels.view(num_classes, 1), 1)

for epoch in range(num_epochs):
    g_loss_epoch = 0
    d_loss_epoch = 0
    
    for batch_idx, (images, labels) in enumerate(train_loader):
        batch_size = images.size(0)

        """
        TODO: Define here your training strategy
        """
        
        d_loss_epoch += d_loss.item()
        g_loss_epoch += g_loss.item()
        
    print('epoch [{}/{}], generator loss:{:.4f}'
          .format(epoch + 1, num_epochs, g_loss_epoch / len(train_loader)))
    g_losses.append(g_loss_epoch/ len(train_loader))
    print('epoch [{}/{}], discriminator loss:{:.4f}'
          .format(epoch + 1, num_epochs, d_loss_epoch / len(train_loader)))
    d_losses.append(d_loss_epoch/ len(train_loader))
    if epoch % sample_interval == 0:
        fake_fixed_images = generator(fixed_noise, fixed_labels_one_hot)
        fake_fixed_images = denorm(fake_fixed_images)
        save_image(fake_fixed_images.cpu().float(), './cGAN/samples_epoch_{}.png'.format(epoch),nrow = 8)
    torch.save(generator.state_dict(), './cGAN/generator.pth')
    torch.save(discriminator.state_dict(), './cGAN/discriminator.pth')
    
np.save('./cGAN/generator_losses.npy', np.array(g_losses))
np.save('./cGAN/discriminator_losses.npy', np.array(d_losses))

### Loss curves

In [0]:
import matplotlib.pyplot as plt
generator_losses = np.load('./cGAN/generator_losses.npy')
plt.plot(list(range(0,generator_losses.shape[0])), generator_losses)
plt.title('Generator Loss')
plt.show()

import matplotlib.pyplot as plt
discriminator_losses = np.load('./cGAN/discriminator_losses.npy')
plt.plot(list(range(0,discriminator_losses.shape[0])), discriminator_losses)
plt.title('Discriminator Loss')
plt.show()

### Sampling

In [0]:
generator.load_state_dict(torch.load("./cGAN/generator.pth"))

"""
TODO: Sample from the noise distribution
"""
